In [ ]:
import h5py
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

## 1. Load Tensor

In [2]:
file_path = "../data/test.h5"
with h5py.File(file_path, 'r') as file:
    data_label = file['label'][...]
        
    tensor_label_test = torch.tensor(data_label)
    
encoded_halflife_test = torch.load('../tensor/encoded_test_halflife_tensor2.pth')

/tmp/ipykernel_222798/1483781461.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoded_halflife_test = torch.load('../tensor/encoded_test_halflife_tensor2.pth')


In [4]:
load_path = '../tensor/tensor_promoter_test_embedding.pth'
tensor_promoter_test = torch.load(load_path)

/tmp/ipykernel_222798/2412120876.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor_promoter_test = torch.load(load_path)


## 2. Define model

In [7]:
class CNNBinaryClassifier(nn.Module):
    def __init__(self):
        super(CNNBinaryClassifier, self).__init__()
        
        self.input_size = 5000
        
        self.conv1 = nn.Conv1d(256, 128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(128, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(64, 16, kernel_size=3, padding=1)  
        self.conv4 = nn.Conv1d(16, 4, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool1d(2, 2)
        
        self.dropout = nn.Dropout(0.5) 
        
        if self.input_size is not None:
            self.fc1 = nn.Linear(self.input_size, 256)
        else:
            self.fc1 = None
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)
        
        self.fc_y = nn.Linear(256,128)
        self.linear_y = nn.Linear(128,1) 
    
    def forward(self, x, y):

        concat_tensor = torch.concat((x,y),dim=1)
        concat_tensor = concat_tensor.permute(0,2,1)
        
        concat_tensor = self.pool(nn.functional.relu(self.conv1(concat_tensor)))
        concat_tensor = self.pool(nn.functional.relu(self.conv2(concat_tensor)))
        concat_tensor = self.pool(nn.functional.relu(self.conv3(concat_tensor)))  
        concat_tensor = self.pool(nn.functional.relu(self.conv4(concat_tensor)))  
        
        concat_tensor = concat_tensor.view(concat_tensor.size(0), -1)# 32,5000
        
        if self.fc1 is None: 
            self.input_size = concat_tensor.size(1)  
            self.fc1 = nn.Linear(self.input_size, 256).to(concat_tensor.device) 
        
        concat_tensor = nn.functional.relu(self.fc1(concat_tensor))
        concat_tensor = self.dropout(concat_tensor)
        concat_tensor = nn.functional.relu(self.fc2(concat_tensor))
        logit = self.fc3(concat_tensor)  
        
        return logit

## 3. Obtain results

In [ ]:
model = CNNBinaryClassifier().to('cpu')

model.load_state_dict(torch.load('../model/CNNHalflifeEncodePred2.pth'))

model.eval()

tensor_halflife_test = encoded_halflife_test.float().to('cpu')
tensor_promoter_test = tensor_promoter_test.float().to('cpu')


with torch.no_grad():
    outputs = model(tensor_halflife_test,tensor_promoter_test)

probabilities = torch.sigmoid(outputs)

valid_labels = tensor_label_test.float()

predicted_labels = (probabilities > 0.55).float() 

# 计算各项指标
accuracy = accuracy_score(valid_labels.numpy(), predicted_labels.numpy())
precision = precision_score(valid_labels.numpy(), predicted_labels.numpy())
recall = recall_score(valid_labels.numpy(), predicted_labels.numpy())
f1 = f1_score(valid_labels.numpy(), predicted_labels.numpy())

auc = roc_auc_score(valid_labels.numpy(), probabilities.numpy())

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'AUC: {auc:.4f}')